In [151]:
import pandas as pd
import numpy as np
import torch
import itertools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso

# Chargement des données
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

# Liste des colonnes qu'on veut garder en valeur numérique : 
numeric_columns = ['OverallQual','OverallCond','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','YrSold']
#mainFeatures = ['OverallQual', 'TotRmsAbvGrd', 'LotArea', 'GrLivArea', 'YrSold']
mainFeatures = ['OverallQual','GrLivArea','YrSold']
target = 'SalePrice'

# Prétraitement des données textuelles : One-Hot Encoding pour les colonnes catégorielles
data_train = pd.get_dummies(data_train, drop_first=True)  # `drop_first=True` pour éviter la multicolinéarité
data_test = pd.get_dummies(data_test, drop_first=True)
data_train, data_test = data_train.align(data_test, join='left', axis=1, fill_value=0)

# Forcer la conversion des colonnes numériques en float
data_train[numeric_columns] = data_train[numeric_columns].astype(float)
data_test[numeric_columns] = data_test[numeric_columns].astype(float)

#On remplace les NaN values par la moyenne de la colonne)
data_train.fillna(data_train.mean(), inplace=True)
data_test.fillna(data_test.mean(), inplace=True)

# Séparation des données en X = Features et y = Target
X = data_train.drop(target, axis=1).values  # Sélectionner les features de cette combinaison
#X = data_train[mainFeatures].values # Sélectionner les features de cette combinaison
y = data_train[target].values
       
# Diviser les données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#On entraine les données sur le Log des prix, pour avoir une distribution des prix moins centrée autour de la moyenne et améliorer l'apprentissage
y_train_log = np.log1p(y_train)  # Transformation log1p (log(x+1)) pour éviter problèmes avec les 0
y_test_log = np.log1p(y_test)

#Standardisation des données (avec le même scaler pour les données de Test et de Train)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#Boucle utilisée pour tester le nombre de features optimal à sélectionner pour mon modèle : 
"""
# Dictionnaire pour stocker les résultats
results = []
i = 5

while i <= 20 :
    
    print(f"Entraînement avec les {i} best features :")
    errors = []  # Liste pour stocker l'erreur de chaque essai (30 essais)

    for a in range(30) :
        selector = RFE(lasso, n_features_to_select=i)  # Sélectionner les i meilleures caractéristiques
        X_train_selected = selector.fit_transform(X_train_scaled, y_train_log)
        X_test_selected = selector.transform(X_test_scaled)
        model_svm.fit(X_train_selected, y_train_log)

        # Faire des prédictions sur les données de test
        y_test_pred_log = model_svm.predict(X_test_selected)

        # Inverser la transformation logarithmique pour obtenir les valeurs réelles
        y_test_pred = np.expm1(y_test_pred_log)  # expm1 inverse log1p

        # Calcul de l'erreur quadratique moyenne
        mse = mean_squared_error(y_test, y_test_pred)
        errMoy = 100 * (mse ** 0.5) * len(y_test) / sum(y_test)

        errors.append(errMoy)

    # Calculer l'erreur moyenne sur les 30 essais
    mean_error = np.mean(errors)

    # Sauvegarder les résultats
    results.append((i, mean_error))

    print(f"ErrMoy moyenne de {mean_error}")

    i+=1

# Trier les résultats par erreur moyenne croissante
results.sort(key=lambda x: x[1])

# Afficher les résultats
print("\nClassement par erreur moyenne en fct du nb de features sélectionnées :")
for i, mean_error in results:
    print(f"Nb features : {i} - Erreur moyenne : {mean_error:.2f}%")  
"""

# Création et entraînement du modèle SVM
model_svm = SVR(kernel='rbf')
lasso = Lasso(alpha=0.01)  # Choisir un alpha faible pour ne pas trop pénaliser les coefficients
selector = RFE(lasso, n_features_to_select=20)  # Sélectionner les 10 meilleures caractéristiques
X_train_scaled = selector.fit_transform(X_train_scaled, y_train_log)
X_test_scaled = selector.transform(X_test_scaled)
model_svm.fit(X_train_scaled, y_train_log)

# Faire des prédictions sur les données de test
y_test_pred_log = model_svm.predict(X_test_scaled)

# Inverser la transformation logarithmique pour obtenir les valeurs réelles
y_test_pred = np.expm1(y_test_pred_log)  # expm1 inverse log1p

 # Calcul de l'erreur quadratique moyenne
mse = mean_squared_error(y_test, y_test_pred)
errMoy = 100 * (mse ** 0.5) * len(y_test) / sum(y_test)

comparison_df = pd.DataFrame({
    'Y Réel': y_test,    # Valeurs réelles (targets)
    'Y Prédit': y_test_pred   # Valeurs prédites
})

print(comparison_df.head())
print(errMoy)

#Application du modèle sur données de validation : 
# Définition et formattage des données X de validation
X_valid = data_test.drop(target, axis=1).values  # Sélectionner les features de cette combinaison
scaler_valid = StandardScaler()
X_valid_scaled = scaler_valid.fit_transform(X_valid)
X_valid_scaled = selector.transform(X_valid_scaled)

#Prédiction à l'aide du modèle créé précédemment
y_valid_pred_log = model_svm.predict(X_valid_scaled)

# Inverser la transformation logarithmique pour obtenir les valeurs réelles
y_valid_pred = np.expm1(y_valid_pred_log)  # expm1 inverse log1p

# Créer un DataFrame avec l'ID et les résultats prédits
predictions_df_svm = pd.DataFrame({
    'Id': data_test['Id'],  # Garder la colonne 'Id' du fichier original
    'SalePrice': y_valid_pred  # Ajouter la colonne 'SalePrice' avec les prédictions
})

# Sauvegarder le DataFrame dans un fichier CSV avec les deux colonnes
predictions_df_svm.to_csv('test_predictions_svm.csv', index=False)

print("Les prédictions SVM ont été sauvegardées dans 'test_predictions_svm.csv'.")


   Y Réel       Y Prédit
0  374000  344327.089818
1  232000  258809.319738
2  173000  181852.155022
3  128500  126236.584265
4  128500  133335.561450
14.84829646884698
Les prédictions SVM ont été sauvegardées dans 'test_predictions_svm.csv'.
